In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 5-5. 糖衣構文

### 5-5-1. 糖衣構文（の多く）も多重ディスパッチ

### 5-5-2. 例

#### 例1. インデクシング関連

##### コード5-28.	GeometricSequence.jl（コード5-20.、コード5-24. 再掲）

In [2]:
struct GeometricSequence{T<:Number} <: AbstractVector{T}
    a::T
    r::T
    n::Int
end

function GeometricSequence(a::T1, r::T2, n::Integer) where {T1<:Number, T2<:Number}
    GeometricSequence(promote(a, r)..., Int(n))
end

Base.length(seq::GeometricSequence) = seq.n
Base.size(seq::GeometricSequence) = (seq.n,)
Base.getindex(seq::GeometricSequence, index::Integer) = seq.a * seq.r ^ (index - 1)

function Base.show(io::IO, seq::GeometricSequence)
    print(io, "GeometricSequence(", seq.a, ", ", seq.r, ", ", seq.n, ")")
end
Base.show(io::IO, ::MIME"text/plain", seq::GeometricSequence) = show(io, seq)

function Base.sum(seq::GeometricSequence)
    seq.a * (1 - seq.r^seq.n) / (1 - seq.r)
end

function Base.sum(seq::GeometricSequence{<:Rational})
    c = numerator(seq.a)
    b = denominator(seq.a)
    q = numerator(seq.r)
    p = denominator(seq.r)
    n = seq.n
    c * (p^n - q^n) // (b * p^(n-1) * (p - q))
end

function Base.sum(seq::GeometricSequence{<:Integer})
    seq.a * ((seq.r^seq.n - 1) ÷ (seq.r - 1))
end

##### コード5-29.	`GeometricSequence` におけるインデクシングの動作例

In [3]:
seq1 = GeometricSequence(1, 0.5, 20);

In [4]:
seq1[1] == getindex(seq1, 1) == 1.0

true

In [5]:
seq1[2] == getindex(seq1, 2) == 0.5

true

In [6]:
seq1[10]

0.001953125

In [7]:
seq1[20]

1.9073486328125e-6

In [8]:
seq1[2:4]

3-element Vector{Float64}:
 0.5
 0.25
 0.125

In [9]:
seq1[begin]

1.0

In [10]:
seq1[end]

1.9073486328125e-6

##### コード5-30.	FreeIndexList.jl

In [11]:
mutable struct FreeIndexList{N<:Number}
    _begin::Int
    _end::Int
    data::Dict{Int, N}

    FreeIndexList{N}() where {N} = new{N}(1, 0, Dict{Int, N}())
    function FreeIndexList(src::AbstractVector{N}) where {N}
        _begin = firstindex(src)
        _end = lastindex(src)
        d = Dict(i=>src[i] for i in _begin:_end)
        new{N}(_begin, _end, d)
    end
end

# getindex()
Base.getindex(fil::FreeIndexList{N}, i) where N = get(fil.data, i, zero(N))
Base.getindex(fil::FreeIndexList, idxs::AbstractVector) = [fil[i] for i in idxs]

# setindex!()
function Base.setindex!(fil::FreeIndexList, v, i)
    if fil._begin > fil._end
        fil._begin = fil._end = i
    else
        fil._begin > i && (fil._begin = i)
        fil._end < i && (fil._end = i)
    end
    fil.data[i] = v
end

# `begin`
Base.firstindex(fil::FreeIndexList) = fil._begin

# `end`
Base.lastindex(fil::FreeIndexList) = fil._end

##### コード5-31.	`FreeIndexList` のインデクシング動作例

In [12]:
fil = FreeIndexList([1, 1, 2, 3, 5])

FreeIndexList{Int64}(1, 5, Dict(5 => 5, 4 => 3, 2 => 1, 3 => 2, 1 => 1))

In [13]:
fil[1]  # == getindex(fil, 1)

1

In [14]:
fil[2:6]  # == getindex(fil, 2:6)

5-element Vector{Int64}:
 1
 2
 3
 5
 0

In [15]:
for i = 6:10
    fil[i] = fil[i-1] + fil[i-2]
end

In [16]:
fil

FreeIndexList{Int64}(1, 10, Dict(5 => 5, 4 => 3, 6 => 8, 7 => 13, 2 => 1, 10 => 55, 9 => 34, 8 => 21, 3 => 2, 1 => 1…))

In [17]:
fil[end]

55

In [18]:
for i = -1:-1:-10
    fil[i] = fil[i+2] - fil[i+1]
end

In [19]:
fil[-10] == fil[begin] == -55

true

In [20]:
fil[begin:end]  # == fil[-10:10]

21-element Vector{Int64}:
 -55
  34
 -21
  13
  -8
   5
  -3
   2
  -1
   1
   0
   1
   1
   2
   3
   5
   8
  13
  21
  34
  55

#### 例2. プロパティ

##### コード5-32.	プロパティ参照の定義・実行例 (1)

In [21]:
struct PropParrot end

In [22]:
Base.getproperty(::PropParrot, name::Symbol) = name

In [23]:
parrot = PropParrot()

PropParrot()

In [24]:
parrot.a

:a

In [25]:
parrot.do

:do

In [26]:
parrot.some_long_name

:some_long_name

In [27]:
parrot.日本語もOK

:日本語もOK

In [28]:
parrot.var"こうすれば スペース区切でも OK!"

Symbol("こうすれば スペース区切でも OK!")

##### コード5-33.	プロパティ参照の定義・実行例 (2)

In [29]:
struct PropParrot2{X, Y}
    x::X
    y::Y
end

In [30]:
function Base.getproperty(parrot2::PropParrot2, name::Symbol)
    hasfield(PropParrot2, name) && return getfield(parrot2, name)
    name
end

In [31]:
parrot2 = PropParrot2(123, "漢字")

PropParrot2{Int64, String}(123, "漢字")

In [32]:
parrot2.x

123

In [33]:
parrot2.y

"漢字"

In [34]:
parrot2.z

:z

In [35]:
parrot2.var"x,y,z"

Symbol("x,y,z")

##### コード5-34.	Future.jl

In [36]:
mutable struct Future
    value
    done::Bool

    function Future()
        object = new()
        # object.done = false
        setfield!(object, :done, false)
        object
    end
    function Future(value)
        new(value, true)
    end
end

function Base.getproperty(f::Future, name::Symbol)
    name === :value || return getfield(f, name)
    f.done || error("the field `value` has not been setteled yet")
    getfield(f, :value)  # `f.value` と書いては絶対ダメ
end

function Base.setproperty!(f::Future, name::Symbol, value)
    name === :done && error("the field `done` cannot be changed manually")
    name === :value || return setfield!(f, name, value)  # この場合はなくても良い
    f.done && error("`value` has already been set.")
    setfield!(f, :done, true)  # `f.done = true` と書いてはいけない
    setfield!(f, :value, value)  # `f.value = value` と書いては絶対ダメ
end

##### コード5-35.	`Future` の動作例

In [37]:
f1 = Future()

Future(#undef, false)

In [38]:
f1.done

false

In [39]:
f1.value

LoadError: the field `value` has not been setteled yet

In [40]:
f1.value = "A String"

"A String"

In [41]:
f1.done

true

In [42]:
f1.value

"A String"

In [43]:
f2 = Future(π)

Future(π, true)

In [44]:
f2.done

true

In [45]:
f2.value

π = 3.1415926535897...

In [46]:
f2.value = "Other Value"

LoadError: `value` has already been set.

In [47]:
f2.done = false

LoadError: the field `done` cannot be changed manually

#### 例3. 後置演算子 `'～`

##### コード5-36.	後置演算子 `'` の多重定義例

In [48]:
let _trtable = Dict(
    'い'=>'こ','こ'=>'い','く'=>'へ','へ'=>'く','し'=>'つ','つ'=>'し','も'=>'や','や'=>'も',
    'キ'=>'サ','サ'=>'キ','シ'=>'ツ','ツ'=>'シ','セ'=>'ヤ','ヤ'=>'セ','ソ'=>'ン','ン'=>'ソ',
    'フ'=>'レ','レ'=>'フ','マ'=>'ム','ム'=>'マ','ラ'=>'ル','ル'=>'ラ'
)
    Base.var"'"(c::AbstractChar) = get(_trtable, c, c)
    # ↓のように書いても同じ
    # Base.adjoint(c::AbstractChar) = get(_trtable, c, c)
end

In [49]:
'A''

'A': ASCII/Unicode U+0041 (category Lu: Letter, uppercase)

In [50]:
'い''

'こ': Unicode U+3053 (category Lo: Letter, other)

In [51]:
['や', 'も', 'つ', 'ル', 'ー', 'メ', 'ソ']'

1×7 adjoint(::Vector{Char}) with eltype Char:
 'も'  'や'  'し'  'ラ'  'ー'  'メ'  'ン'

##### コード5-37.	後置演算子 `'～` の定義例

In [52]:
const var"'⁺" = transpose  # `'⁺` は `'\^+` + TAB で変換可能

transpose (generic function with 28 methods)

In [53]:
A = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [54]:
A'

2×2 adjoint(::Matrix{Int64}) with eltype Int64:
 1  3
 2  4

In [55]:
A'⁺

2×2 transpose(::Matrix{Int64}) with eltype Int64:
 1  3
 2  4

In [56]:
var"'²"(x) = x^2  # `'²` は `'\^2` + TAB で変換可能

'² (generic function with 1 method)

In [57]:
A'²

2×2 Matrix{Int64}:
  7  10
 15  22

In [58]:
A'²'²'²  # == ((A^2)^2)^2

2×2 Matrix{Int64}:
 165751  241570
 362355  528106

In [59]:
var"'₂"(x) = string(x, base=2)  # `'₂` は `'\_2` + TAB で変換可能

'₂ (generic function with 1 method)

In [60]:
10'₂

"1010"

In [61]:
123'₂

"1111011"